In [2]:
import cma_gui as cma
import numpy as np
import pandas as pd

from datetime import datetime
from dateutil.relativedelta import relativedelta
from scipy import stats

In [3]:
# 20 years of data dates
last_date = datetime.strptime(cma.val_dict['as_of_date'], '%m-%d-%Y')
first_date = last_date - relativedelta(years=20) + relativedelta(months=1)

# Functions

In [4]:
def returns_dataframe(file_suffix):
    """ Combine all return streams and combine into one dataframe for beta backfill calculations """
    
    file = r"P:\\Advisory\\Research\\Automation\\CMAs\\Data\\bloomberg_data_" + file_suffix + ".xlsx"
    
    df_equity = pd.read_excel(file, sheet_name='equity_returns', index_col=0)
    df_fixed = pd.read_excel(file, sheet_name='fixed_returns', index_col=0)
    df_alts = pd.read_excel(file, sheet_name='alts_returns', index_col=0)

    # Combine all index values into single dataframe
    df_returns = df_equity.join(df_fixed, how="outer").join(df_alts, how="outer")
    
    # Reorder
    order_name = list(filter(None, list({k:v for (k,v) in cma.val_dict.items() if '_' + file_suffix + '_name' in k}.values())))
    df_returns = df_returns.reindex(columns=order_name)

    # Filter dataframe to only include last 20 years
    df_returns = df_returns.loc[first_date:last_date, :]
    
    return df_returns

In [5]:
def backfill_determination(df_returns, file_suffix):
    """ Determine what index to use for beta calculations """
    
    asset_name = list(filter(None, list({k:v for (k,v) in cma.val_dict.items() if '_' + file_suffix + '_name' in k}.values())))
    asset_beta = list(filter(None, list({k:v for (k,v) in cma.val_dict.items() if '_' + file_suffix + '_beta' in k}.values())))
    df_backfill = pd.DataFrame(list(zip(asset_name, asset_beta)), columns =['Asset Class', 'Beta Relative To']).set_index('Asset Class') 
    
    # Strip out values that don't need backfilled
    filter_vals = ['Building Blocks', 'N/A']
    df_backfill = df_backfill[~df_backfill['Beta Relative To'].str.contains('|'.join(filter_vals))]
    
    return df_backfill

In [6]:
def backfill_calc(df_returns, df_backfill):
    """ Fill in missing data with beta calculated proxy value """
    
    # Determine columns with NaN
    list_nan = df_returns.columns[df_returns.isna().any()].tolist()
    
    # Backfill indexes with beta adjusted returns if history is not available
    for i in range(len(list_nan)):
        list_nan_sub = list_nan[i]
        backfill_sub = df_backfill.loc[list_nan[i], 'Beta Relative To']
        df_bfill = df_returns[[list_nan_sub, backfill_sub]].dropna()
        
        slope, intercept, r_value, p_value, std_err = stats.linregress(df_bfill[backfill_sub].values, df_bfill[list_nan_sub].values)

        df_returns[list_nan_sub].fillna(df_returns[backfill_sub] * slope + intercept, inplace=True)

# USD

In [7]:
df_returns_us = returns_dataframe('us')
df_backfill_us = backfill_determination(df_returns_us, 'us')
backfill_calc(df_returns_us, df_backfill_us)

df_returns_us.to_csv(
    r"P:\\Advisory\\Research\\Automation\\CMAs\\Data\\combined_returns_us.csv")

# Non-USD

In [9]:
# Import index values
df_equity_nonus = pd.read_excel(
    "P:\\Advisory\\Research\\Automation\\CMAs\\Data\\bloomberg_data_nonus.xlsx", sheet_name='equity_returns', index_col=0)
df_fixed_nonus = pd.read_excel(
    "P:\\Advisory\\Research\\Automation\\CMAs\\Data\\bloomberg_data_nonus.xlsx", sheet_name='fixed_returns', index_col=0)
df_alts_nonus = pd.read_excel(
    "P:\\Advisory\\Research\\Automation\\CMAs\\Data\\bloomberg_data_nonus.xlsx", sheet_name='alts_returns', index_col=0)
df_currency = pd.read_excel(
    "P:\\Advisory\\Research\\Automation\\CMAs\\Data\\bloomberg_data_nonus.xlsx", sheet_name='currencies', index_col=0)

# Combine all index values into single dataframe
df_index_nonus = df_equity_nonus.join(df_fixed_nonus, how="outer").join(df_alts_nonus, how="outer")

In [11]:
# Adjust indices for currency
df_index_local= df_index_nonus.mul(df_currency[cma.val_dict['currency']], axis=0)

# Add back in alts and us equity with non local results
df_equity_nonus_usequity = df_equity_nonus['U.S. Equity']
df_alts_nonus_nolocal = df_alts_nonus.merge(df_equity_nonus_usequity, left_index=True, right_index=True)
df_alts_nonus_nolocal = df_alts_nonus_nolocal.add_prefix('USD_')

df_index_local = df_index_local.join(df_alts_nonus_nolocal, how="outer")

# Calculate monthly returns in local currency
df_returns_nonus = df_index_local.pct_change()

# Filter dataframe to only include last 20 years
df_returns_nonus = df_returns_nonus.loc[first_date:last_date, :]

In [13]:
df_backfill_nonus = backfill_determination(df_returns_nonus, 'nonus')
backfill_calc(df_returns_nonus, df_backfill_nonus)
df_returns_nonus

,U.S. Equity,U.S. Small Cap Equity,Europe Ex-UK Equity,Europe Small Cap Equity,UK Equity,Japan Equity,Japan Small Cap Equity,Developed Market Pacific Ex-Japan Equity,Global Emerging Markets Equity,Global Developed Market Sovereign Fixed Income,...,Global High Yield Fixed Income,Emerging Markets Local Currency Fixed Income,Emerging Markets Sovereign Fixed Income USD,Emerging Markets Corporate Fixed Income,Emerging Debt Agg USD,Long-Short Equity,Global Macro,USD_Long-Short Equity,USD_Global Macro,USD_U.S. Equity
1999-01-31,0.052224,0.028345,0.003900,-0.024299,-0.001468,0.016443,0.029761,0.016272,-0.007316,0.004882,...,0.010624,-0.007150,-0.007524,-0.009121,-0.007515,0.059209,0.017135,0.049800,0.008100,0.042877
1999-02-28,-0.001931,-0.064791,-0.019769,0.015674,0.049271,0.004119,0.015395,0.016173,0.036469,-0.004783,...,0.027607,0.030531,0.039898,0.038538,0.039512,0.001746,0.013756,-0.024100,-0.012400,-0.027682
1999-03-31,0.035994,-0.000176,-0.003414,0.012051,0.025399,0.132478,0.128623,0.073344,0.125442,-0.005039,...,0.022251,0.037335,0.048461,0.047143,0.048003,0.034664,0.005031,0.040500,0.010700,0.041838
1999-04-30,0.037140,0.076875,0.027143,0.060468,0.040034,0.043228,0.074507,0.167040,0.125170,-0.000151,...,0.036865,0.045364,0.058566,0.057298,0.058024,0.053857,0.039939,0.052500,0.038600,0.035804
1999-05-31,-0.018832,0.039752,-0.039987,-0.013450,-0.051265,-0.052130,-0.062757,-0.066651,-0.001497,-0.011506,...,-0.019404,-0.027458,-0.033082,-0.034805,-0.032859,0.016599,-0.004693,0.012200,-0.009000,-0.023077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-08-31,0.046267,0.057567,-0.010002,0.001726,-0.029609,0.015030,0.006377,-0.005348,-0.014355,0.012356,...,0.002263,-0.016723,-0.010137,0.002773,-0.000899,0.016950,0.017448,0.004166,0.004658,0.033114
2018-09-30,-0.000765,-0.024132,-0.006753,-0.025562,0.012115,0.026181,0.009335,-0.011789,-0.010263,-0.017014,...,0.008313,0.008276,0.015453,0.004013,0.007877,-0.011350,-0.008546,-0.006069,-0.003250,0.004573
2018-10-31,-0.050026,-0.082898,-0.060380,-0.079685,-0.048088,-0.065433,-0.075638,-0.068522,-0.067965,0.011720,...,0.002261,0.008859,0.000624,0.009556,0.006899,-0.026834,0.000998,-0.046707,-0.019444,-0.069426
2018-11-30,0.020394,0.019632,-0.005376,-0.033974,-0.015651,0.004900,0.028138,0.029992,0.042212,0.005277,...,-0.008656,0.023573,-0.001416,-0.000688,-0.000706,-0.001041,-0.004388,-0.001933,-0.005277,0.019483


In [14]:
df_returns_nonus.to_csv(
    r"P:\\Advisory\\Research\\Automation\\CMAs\\Data\\combined_returns_nonus.csv")